In [1]:
import pandas as pd
import numpy as np

Json Data

In [73]:
import nltk
from nltk.corpus import stopwords
stop_word = stopwords.words('english')
from cleantext import clean
import re
from bs4 import BeautifulSoup

def textCleaning(text):      
                            
        #     bs4
        soup = BeautifulSoup(text, 'lxml')
        text = soup.text
        #     remove urls
        text = re.sub(r'http\S+', " ", text)

        #     remove mentions
        text = re.sub(r'@\w+',' ',text)

        #     remove hastags
        text = re.sub(r'#\w+', ' ', text)
        
        # take alphanumerical
        text = re.sub(r'\W+', ' ', text)

        #     remove digits
        text = re.sub(r'\d+', ' ', text)

        #     remove html tags
        text = re.sub('r<.*?>',' ', text)
        
        #     remove stop words 
        text = text.split()
        text = " ".join([word for word in text if not word in stop_word])    
        
        return text

In [47]:
import json
def read_data(path):
    
    f = open(path)
    data = json.load(f)
    li_title = []
    li_desc = []
    li_query = []
    for i in range(len(data['jobs'])):
        li_title.append(data['jobs'][i]['job_title_by_google'])
        li_desc.append(data['jobs'][i]['desc'])
        li_query.append(data['jobs'][i]['search_query'])
    df = pd.DataFrame(
        {
        'job_roles' : li_query, 'desc' : li_desc, 'title': li_title
        }
    )
    df = df[df['desc'] !='No Information Found']
    return df

In [48]:
def extract_job(data):
  category = 'jobs'
  index = data.find(category)
  return data[:index].strip()

In [49]:
def check_duplicates_in_job_role(df):
    for i in df['job_roles'].unique():
        total = df['desc'][df['job_roles']==i].count()
        duplicated = df['desc'][df['job_roles']==i].duplicated().sum()
        val_40 = int(0.4 * total)
        if duplicated >= val_40 and duplicated > 0:
            print(f'Job role : {i} has more than 40% duplicate values, total : {total}, duplicated : {duplicated} ')

In [50]:
def check_duplicates_among_job_roles(df):
    li_job_roles = df['job_roles'].unique()
    for i in range(len(li_job_roles)):
        j = i + 1
        data1 = df['desc'][df['job_roles']==li_job_roles[i]]
        for j in range(j,len(li_job_roles),1):
            data2 = df['desc'][df['job_roles']==li_job_roles[j]]
            data = pd.concat([data1,data2])
            dup_sum = data.duplicated().sum()
            if dup_sum > 0:
                print(f'Number of duplicate values between {li_job_roles[i]} and {li_job_roles[j]} is/are : {dup_sum}')


In [54]:
def main(json_file_path):
    
    df = read_data(json_file_path)
    df['job_roles'] = df['job_roles'].apply(lambda x : extract_job(x))
    check_duplicates_in_job_role(df)
    df.drop_duplicates(subset='desc', keep='first', inplace=True)
    check_duplicates_among_job_roles(df)
    return df

In [55]:
json_file_path = '/home/user/py/pipeline_job_role/data/transportation_dec_06.json'
df_1 = main(json_file_path)

Job role : Service Technician has more than 40% duplicate values, total : 75, duplicated : 39 
Job role : Taxi Driver has more than 40% duplicate values, total : 9, duplicated : 3 


In [56]:
json_file_path = '/home/user/py/pipeline_job_role/data/transportation_dec_07.json'
df_2 = main(json_file_path)

Job role : Service Technician has more than 40% duplicate values, total : 76, duplicated : 35 


In [57]:
li_jobs_1 = df_1['job_roles'].unique()
len(li_jobs_1)

40

In [58]:
li_jobs_2 = df_1['job_roles'].unique()
len(li_jobs_2)

40

In [62]:
for i in range(len(li_jobs_1)):
    data1 = df_1['desc'][df_1['job_roles']==li_jobs_1[i]]
    data2 = df_2['desc'][df_2['job_roles']==li_jobs_1[i]]
    data = pd.concat([data1,data2])
    df_1_count = data1.count()
    df_2_count = data2.count()
    dup = data.duplicated().sum()
    if dup > 0:
        print(f' Number of duplicate values for {li_jobs_1[i]} between df_1({df_1_count}) and df_2({df_2_count}) is/are : {dup}')

 Number of duplicate values for Railcar Switcher between df_1(8) and df_2(32) is/are : 4
 Number of duplicate values for Motor Vehicle Emissions Inspector between df_1(26) and df_2(10) is/are : 10
 Number of duplicate values for Service Technician between df_1(36) and df_2(39) is/are : 23
 Number of duplicate values for Airline Pilot between df_1(8) and df_2(14) is/are : 1
 Number of duplicate values for Automotive Master Mechanic between df_1(20) and df_2(2) is/are : 1
 Number of duplicate values for Aviation Inspector between df_1(6) and df_2(15) is/are : 3
 Number of duplicate values for Bus Driver between df_1(6) and df_2(10) is/are : 5
 Number of duplicate values for Captain between df_1(42) and df_2(2) is/are : 2
 Number of duplicate values for Dispatcher between df_1(8) and df_2(8) is/are : 5
 Number of duplicate values for Repairer between df_1(1) and df_2(8) is/are : 1
 Number of duplicate values for Truck Driver between df_1(8) and df_2(24) is/are : 8
 Number of duplicate val

In [64]:
from sklearn.utils import shuffle

df_combined = pd.concat([df_1,df_2])
df_combined = shuffle(df_combined).reset_index(drop=True)
df_combined.shape

(1286, 3)

In [74]:
df_combined['clean_desc'] = df_combined['desc'].apply(lambda x: textCleaning(x))
df_combined.head(2)

,job_roles,desc,title,clean_desc
0,Railcar Switcher,•\nStable work history with good references fr...,Rail Switchman / Locomotive Engineer,Stable work history good references past emplo...
1,Attendant,Detailed oriented and able to handle multiple ...,Housekeeping Room Attendant,Detailed oriented able handle multiple tasks H...


In [79]:
df_combined.drop_duplicates(subset='clean_desc', keep='first', inplace=True)
df_combined.shape

(1111, 4)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

plt.figure(figsize=(25,5))
sns.countplot(df_combined['job_roles'])
plt.xticks(rotation=90);

In [80]:
from collections import Counter
count = Counter(df_combined['job_roles'])
li_job=[]
li_value=[]
for i in count:
    if count[i] <= 10:
        li_job.append(i)
        li_value.append(count[i])

In [81]:
df_less = pd.DataFrame()
df_less['Job_roles'] = li_job
df_less['Value'] = li_value
df_less.head()

,Job_roles,Value
0,Track Switch Repairer,7
1,Ship and Boat Captain,10
2,Operations Technologist,7
3,Railroad Car Inspector,10
4,Inventory Clerk,10


In [82]:
df_combined[['job_roles','title','clean_desc']].to_csv('transport_combined_dec_07.csv', index = False)

In [83]:
df_less.to_csv('transport_combined_dec_07_jobs_less_10.csv', index = False)